<a href="https://colab.research.google.com/github/XZhangNH/XZhangNH/blob/Projects/neuromatch_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pkg_resources
import os
import warnings
warnings.filterwarnings(action='once')

In [ ]:
!git clone --recurse-submodules https://github.com/ColeLab/ActflowToolbox.git

fatal: destination path 'ActflowToolbox' already exists and is not an empty directory.


In [ ]:
!pip install nltools

     |████████████████████████████████| 3.3MB 4.9MB/s 
     |████████████████████████████████| 4.9MB 44.4MB/s 


/usr/local/lib/python3.7/dist-packages/google/colab/_pip.py:87: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.7/dist-packages/deepdish-0.3.6.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):
/usr/local/lib/python3.7/dist-packages/google/colab/_pip.py:87: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.7/dist-packages/nilearn-0.8.0.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):
/usr/local/lib/python3.7/dist-packages/google/colab/_pip.py:87: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.7/dist-packages/nltools-0.4.4.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in open(toplevel):
/usr/local/lib/python3.7/dist-packages/google/colab/_pip.py:87: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.7/dist-packages/pynv-0.2.dist-info/top_level.txt' mode='r' encoding='UTF-8'>
  for line in 

In [ ]:
import ActflowToolbox as actflow
from nltools.utils import get_resource_path
from nltools.file_reader import onsets_to_dm
from nltools.data import Design_Matrix
import statsmodels.api as sm
from sklearn.preprocessing import scale

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(

In [ ]:
import urllib.request
import tarfile

In [ ]:
thetarfile = "https://osf.io/s4h8j/download/"
ftpstream = urllib.request.urlopen(thetarfile)
thetarfile = tarfile.open(fileobj=ftpstream, mode="r|gz")
thetarfile.extractall()

In [ ]:
thetarfile = "https://osf.io/bqp7m/download/"
ftpstream = urllib.request.urlopen(thetarfile)
thetarfile = tarfile.open(fileobj=ftpstream, mode="r|gz")
thetarfile.extractall()

In [ ]:
fname = "hcp_task.tgz"
if not os.path.exists(fname):
  !wget -qO $fname https://osf.io/2y3fw/download
  !tar -xzf $fname -C $HCP_DIR --strip-components=1

tar (child): -C: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now


In [ ]:
# The download cells will store the data in nested directories starting here:
HCP_DIR = "./hcp"
if not os.path.isdir(HCP_DIR):
  os.mkdir(HCP_DIR)
  
# The data shared for NMA projects is a subset of the full HCP dataset
N_SUBJECTS = 339

# The data have already been aggregated into ROIs from the Glasesr parcellation
N_PARCELS = 360

# The acquisition parameters for all tasks were identical
TR = 0.72  # Time resolution, in sec

# The parcels are matched across hemispheres with the same order
HEMIS = ["Right", "Left"]

# Each experiment was repeated multiple times in each subject
N_RUNS_REST = 4
N_RUNS_TASK = 2

# Time series data are organized by experiment, with each experiment
# having an LR and RL (phase-encode direction) acquistion
BOLD_NAMES = [
  "rfMRI_REST1_LR", "rfMRI_REST1_RL",
  "rfMRI_REST2_LR", "rfMRI_REST2_RL",
  "tfMRI_MOTOR_RL", "tfMRI_MOTOR_LR",
  "tfMRI_WM_RL", "tfMRI_WM_LR",
  "tfMRI_EMOTION_RL", "tfMRI_EMOTION_LR",
  "tfMRI_GAMBLING_RL", "tfMRI_GAMBLING_LR",
  "tfMRI_LANGUAGE_RL", "tfMRI_LANGUAGE_LR",
  "tfMRI_RELATIONAL_RL", "tfMRI_RELATIONAL_LR",
  "tfMRI_SOCIAL_RL", "tfMRI_SOCIAL_LR"
]



#Dictionaries 
conditions_dict={
    "motor": ["cue", "rf", "lf", "rh", "lh"],
    "wm": ["0bk_body", "0bk_faces", "0bk_places", "0bk_tools", "2bk_body", 
           "2bk_faces", "2bk_places", "2bk_tools"],
    "emotion": ["fear", "neut"],
    "gambling": ["win", "loss"],
    "language": ["story", "math"],
    "relational": ["match", "relation"],
    "social": ["mental", "rnd"]}

run_length_dict = {
    "motor": 284,
    "wm": 405,
    "emotion": 176,
    "gambling": 253,
    "language": 316,
    "relational": 232,
    "social": 274}

bold_name_dict = {
    "rest": ["rfMRI_REST1_LR", "rfMRI_REST1_RL", "rfMRI_REST2_LR", "rfMRI_REST2_RL"],
    "motor": ["tfMRI_MOTOR_RL", "tfMRI_MOTOR_LR"],
    "wm": ["tfMRI_WM_RL", "tfMRI_WM_LR"],
    "emotion": ["tfMRI_EMOTION_RL", "tfMRI_EMOTION_LR"],
    "gambling": ["tfMRI_GAMBLING_RL", "tfMRI_GAMBLING_LR"],
    "language": ["tfMRI_LANGUAGE_RL", "tfMRI_LANGUAGE_LR"],
    "relational": ["tfMRI_RELATIONAL_RL", "tfMRI_RELATIONAL_LR"],
    "social": ["tfMRI_SOCIAL_RL", "tfMRI_SOCIAL_LR"]}

task_run_dict = {
    "rest": [1,2,3,4],
    "motor": [5,6],
    "wm": [7,8],
    "emotion": [9,10],
    "gambling": [11,12],
    "language": [13, 14],
    "relational": [15, 16],
    "social": [17, 18]}

In [ ]:
def get_cond_evs(cond_name, task_name, subject, run = 1):
  """Load onset files for a single condition from a task.
  
  Args:
    cond_name (str): condition name pulled from the conditions_dict for task_name
    task_name (str): task name instead of the bold run
    subject (int): 0-based subject ID to load
    run (int): 1 or 2 for task runs (use run-1 for indexing)

  Returns
    cond_evs (n_blocks x 3): Events file for single condition to be formatted for design matrix

  """
  bold_name = bold_name_dict[task_name][run-1]
  cond_evs = pd.read_csv('%s/subjects/%s/%s/%s/EVs/%s.txt'%(HCP_DIR, subject, EXPERIMENTS, bold_name, cond_name), sep="\t", header=None)
  cond_evs = cond_evs.rename(columns={0: "Onset", 1: "Duration", 2: "amplitude"})
  cond_evs = cond_evs.drop(columns=['amplitude'])
  cond_evs['Stim'] = cond_name

  return cond_evs

def get_run_evs(subject, task_name, run = 1):
  """Load onset files for a full file.
  
  Args:
    task_name (str): task name instead of the bold run
    subject (int): 0-based subject ID to load
    run (int): 1 or 2 for task runs (use run-1 for indexing)

  Returns
    evs (n_blocks for run x 3 array): Events file for single condition to be formatted for design matrix

  """

  conditions = conditions_dict[task_name]

  evs = pd.DataFrame()

  for cond in conditions:
    cond_evs = get_cond_evs(cond, task_name, subject, run)
    evs = evs.append(cond_evs)
  
  evs = evs.sort_values(by="Onset") 

  return evs

def run_evs_to_dm(run_evs, task_name, TR=.72, convolve = True, add_poly = 2, dct_basis=False):

  sampling_freq = 1./TR
  run_length = run_length_dict[task_name]
  dm = onsets_to_dm(run_evs, sampling_freq=sampling_freq, run_length=run_length, sort=True, add_poly=add_poly)

  if convolve: 
    dm = dm.convolve()

  if dct_basis:
    dm = dm.add_dct_basis()

  return dm

def get_task_dms(subject, task_name, TR = .72, convolve = True, add_poly = 2, dct_basis=False):

  runs = list(range(1,len(task_run_dict[task_name])+1))
  task_dm = Design_Matrix(sampling_freq=1./TR)

  for run in runs:
    run_evs = get_run_evs(subject=subject, task_name=task_name, run=run)
    run_dm = run_evs_to_dm(run_evs=run_evs, task_name=task_name, add_poly=add_poly, dct_basis=dct_basis)
    task_dm = task_dm.append(run_dm)

  return task_dm

def load_run_timeseries(subject, task_name, run = 1, remove_mean=True, scale_ts=True):
  """Load timeseries data for a single subject and single run.
  
  Args:
    subject (int): 0-based subject ID to load
    task_name (str): task name instead of the bold run
    run (int): 1 or 2 for task runs
    remove_mean (bool): If True, subtract the parcel-wise mean

  Returns
    ts (n_parcel x n_timepoint array): Array of BOLD data values

  """
  bold_run = task_run_dict[task_name][run-1]

  bold_path = f"{HCP_DIR}/subjects/{subject}/timeseries"
  bold_file = f"bold{bold_run}_Atlas_MSMAll_Glasser360Cortical.npy"
  ts = np.load(f"{bold_path}/{bold_file}")
  if remove_mean:
    ts -= ts.mean(axis=1, keepdims=True)

  if scale_ts:
    #scales each parcel's timeseries (instead of scaling the bold for one 1 TR from all parcels)
    ts = scale(ts, axis=1)
  return ts

def load_task_timeseries(subject, task_name, remove_mean=True, scale_ts = True):
  
  runs = list(range(1,len(task_run_dict[task_name])+1))
  task_ts = np.empty((360, 0))

  for run in runs:
    #since everything is loaded by run and scale_ts is true each parcel should be 
    #scaled for each parcel and for each run separately before being concatenated together
    cur_run_ts = load_run_timeseries(subject=subject, task_name=task_name, run=run)
    task_ts = np.append(task_ts, cur_run_ts, axis=1)
  
  return task_ts

def get_sub_task_resids(subject, task_name):
 
  #load task data
  task_ts = load_task_timeseries(subject=subject, task_name=task_name)

  #make design matrix
  task_dm = get_task_dms(subject=subject, task_name=task_name)

  #initialize empty variables to store data in
  run_length = run_length_dict[task_name]
  num_runs = len(task_run_dict[task_name])
  resids = np.empty((0, num_runs*run_length))

  #loop through parcels, run regression and extract residuals
  for parcel in range(len(task_ts)):
    model = sm.OLS(task_ts[parcel], task_dm)
    results = model.fit()
    cur_resids = np.array([results.resid])
    resids = np.append(resids, cur_resids, axis=0)

  #store parcel residuals in same format as original BOLD
  out_dir = './hcp/residuals/%s'%(task_name)
  if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

  out_fn = '%s_%s_Glasser360Cortical.npy'%(task_name, str(subject))

  np.save(os.path.join(out_dir, out_fn), resids)

  return resids

def load_fcs(task_name, fc_type):
  
  base_dir = '/content/drive/My Drive/Colab Notebooks'

  if fc_type == "resid":
    fc_dir = os.path.join(base_dir, 'residual_fcs')
  
  elif fc_type == "task":
    fc_dir = os.path.join(base_dir, 'task_preds_fcs')

  elif fc_type == "rest":
    fc_dir = os.path.join(base_dir, 'rest_fcs')

  input_dir = os.path.join(fc_dir, task_name)
  fcs_list = os.listdir(input_dir)
  fcs = np.zeros((360, 360, len(fcs_list)))

  for i, fc in enumerate(fcs_list):
    fcs[:,:,i] = np.load(os.path.join(input_dir, fc))

  return fcs

def get_sub_task_pred(subject, task_name):
 
  #load task data
  task_ts = load_task_timeseries(subject=subject, task_name=task_name)

  #make design matrix
  task_dm = get_task_dms(subject=subject, task_name=task_name)
  task_regs = task_dm.iloc[:,:len(conditions_dict[task_name])]

  #initialize empty variables to store data in
  run_length = run_length_dict[task_name]
  num_runs = len(task_run_dict[task_name])
  preds = np.empty((0, num_runs*run_length))

  #loop through parcels, run regression and extract residuals
  for parcel in range(len(task_ts)):
    model = sm.OLS(task_ts[parcel], task_dm)
    results = model.fit()
    task_coefs = results.params[:len(conditions_dict[task_name])]
    cur_preds = np.zeros(num_runs*run_length)
    for i in range(len(conditions_dict['emotion'])):
      cur_preds += task_coefs[i]*task_regs.iloc[:,i]
    cur_preds = np.array(cur_preds).reshape(1, -1)
    preds = np.append(preds, cur_preds, axis=0)

  #store parcel residuals in same format as original BOLD
  out_dir = './hcp/task_preds/%s'%(task_name)
  if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

  out_fn = '%s_%s_Glasser360Cortical.npy'%(task_name, str(subject))

  np.save(os.path.join(out_dir, out_fn), preds)

  return preds

In [ ]:
regions = np.load('./hcp_task/regions.npy').T
region_info = dict(
    name=regions[0].tolist(),
    network=regions[1],
    myelin=regions[2].astype(np.float),
)

In [ ]:
networkpartition_dir = pkg_resources.resource_filename('ActflowToolbox.dependencies', 'ColeAnticevicNetPartition/')
networkdef = np.loadtxt(networkpartition_dir + '/cortex_parcel_network_assignments.txt')
networkorder = np.asarray(sorted(range(len(networkdef)), key=lambda k: networkdef[k]))
networkorder.shape = (len(networkorder),1)
netorder=networkorder[:,0]

In [ ]:
network_dict = {1: 'Visual1', 
       2: 'Visual2', 
       3:'Somatomotor',
       4:'Cingulo-Oper',
       5:'Language',
       6:'Default',
       7:'Frontopariet',
       8:'Auditory',
       9:'Posterior-Mu',
       10:'Dorsal-atten',
       11:'Ventral-Mult',
       12:'Orbito-Affec'}

In [ ]:
net_col_dict = {'Visual1': (0, 0, 1),
                'Visual2':(0.3922, 0, 1),
                'Somatomotor':(0, 1, 1),
                'Cingulo-Oper':(0.6, 0, 0.6),
                'Language':(0, 1, 0),
                'Default':(0, 0.6, 0.6),
                'Frontopariet':(1, 1, 0),
                'Auditory':(0.98, 0.24, 0.98),
                'Posterior-Mu':(1, 0, 0),
                'Dorsal-atten':(0.7, 0.35, 0.16),
                'Ventral-Mult':(1, 0.6, 0),
                'Orbito-Affec':(0.25, 0.5, 0)}

In [ ]:
subject = 100307
task_name = 'relational'
run = 1
EXPERIMENTS = 'RELATIONAL'

In [ ]:
run_evs = get_run_evs(subject=subject, task_name = task_name, run = run)
run_evs

FileNotFoundError: ignored

In [ ]:
dm = run_evs_to_dm(run_evs, task_name = task_name)
dm.heatmap()

In [ ]:
dm_c = dm.convolve()
dm_c.heatmap()

In [ ]:
bold_data = load_run_timeseries(subject=subject, task_name=task_name, run=run)

In [ ]:
plt.plot(scale(bold_data[0]), label = "Scaled BOLD from parcel 0")
plt.plot(dm_c['fear_c0'], label = "Fear condition regressor")
plt.plot(dm_c['neut_c0'], label = "Neut condition regressor")
plt.legend()

In [ ]:
mod = sm.OLS(scale(bold_data[0]), dm_c)
res = mod.fit()

In [ ]:
res.summary()